In [1]:
import numpy as np
import cv2

def order_points(pts):
    # initialzie a list of coordinates that will be ordered
    # such that the first entry in the list is the top-left,
    # the second entry is the top-right, the third is the
    # bottom-right, and the fourth is the bottom-left
	rect = np.zeros((4, 2), dtype = "float32")

	# the top-left point will have the smallest sum, whereas
	# the bottom-right point will have the largest sum
	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[2] = pts[np.argmax(s)]

	# now, compute the difference between the points, the
	# top-right point will have the smallest difference,
	# whereas the bottom-left will have the largest difference
	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[3] = pts[np.argmax(diff)]

	# return the ordered coordinates
	return rect

def four_point_transform(image, pts):
	# obtain a consistent order of the points and unpack them
	# individually
	rect = order_points(pts)
	(tl, tr, br, bl) = rect

	# compute the width of the new image, which will be the
	# maximum distance between bottom-right and bottom-left
	# x-coordiates or the top-right and top-left x-coordinates
	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))

	# compute the height of the new image, which will be the
	# maximum distance between the top-right and bottom-right
	# y-coordinates or the top-left and bottom-left y-coordinates
	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))

	# now that we have the dimensions of the new image, construct
	# the set of destination points to obtain a "birds eye view",
	# (i.e. top-down view) of the image, again specifying points
	# in the top-left, top-right, bottom-right, and bottom-left
	# order
	dst = np.array([
		[0, 0],
		[maxWidth - 1, 0],
		[maxWidth - 1, maxHeight - 1],
		[0, maxHeight - 1]], dtype = "float32")

	# compute the perspective transform matrix and then apply it
	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

	# return the warped image
	return warped

In [2]:
from PIL import Image
from PIL import ImageFilter
im = Image.open("fake_form.jpg")

In [3]:
%tb
# USAGE
# python scan.py --image images/page.jpg

from skimage.filters import threshold_local
import numpy as np
import cv2
import imutils

# load the image and compute the ratio of the old height
# to the new height, clone it, and resize it
image = cv2.imread("example_01.png",0)
# ratio = image.shape[0] / 500
# orig = image.copy()
# image = imutils.resize(image, height = 500)

# convert the image to grayscale, blur it, and find edges
# in the image
image = cv2.GaussianBlur(image, (5, 5), 0)
edged = cv2.Canny(image, 75, 200)

# show the original image and the edge detected image
print("STEP 1: Edge Detection")
# image.show()
# edged.show()
# cv2.imshow("Image", image)
# cv2.imshow("Edged", edged)

# find the contours in the edged image, keeping only the
# largest ones, and initialize the screen contour
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if imutils.is_cv2() else cnts[1]
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
screenCnt = cnts[0:4]
print(screenCnt)
# loop over the contours
for c in cnts:
	# approximate the contour
	peri = cv2.arcLength(c, True)
	approx = cv2.approxPolyDP(c, 0.02 * peri, True)

	# if our approximated contour has four points, then we
	# can assume that we have found our screen
	if len(approx) == 4:
		screenCnt = approx
		break

# show the contour (outline) of the piece of paper
print("STEP 2: Find contours of paper")
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)

# apply the four point transform to obtain a top-down
# view of the original image
warped = four_point_transform(image, screenCnt.reshape(4, 2))

# convert the warped image to grayscale, then threshold it
# to give it that 'black and white' paper effect
T = threshold_local(warped, 11, offset = 10, method = "gaussian")
warped = (warped > T).astype("uint8") * 255

# show the original and scanned images
print("STEP 3: Apply perspective transform")
cv2.imshow("Original", image)
cv2.imshow("Scanned", warped)

No traceback available to show.


STEP 1: Edge Detection
[array([[[351, 116]],

       [[350, 117]],

       [[349, 117]],

       ...,

       [[361, 123]],

       [[361, 122]],

       [[355, 116]]], dtype=int32), array([[[351, 117]],

       [[352, 116]],

       [[354, 116]],

       ...,

       [[348, 118]],

       [[349, 118]],

       [[350, 117]]], dtype=int32), array([[[296, 216]],

       [[295, 217]],

       [[294, 217]],

       [[292, 219]],

       [[290, 217]],

       [[286, 217]],

       [[281, 222]],

       [[280, 222]],

       [[280, 224]],

       [[279, 225]],

       [[279, 229]],

       [[280, 230]],

       [[280, 232]],

       [[281, 233]],

       [[281, 236]],

       [[283, 238]],

       [[283, 239]],

       [[284, 240]],

       [[284, 241]],

       [[285, 242]],

       [[285, 243]],

       [[288, 246]],

       [[290, 246]],

       [[291, 247]],

       [[293, 247]],

       [[294, 246]],

       [[296, 246]],

       [[298, 244]],

       [[299, 244]],

       [[304, 239]],